<a href="https://colab.research.google.com/github/ZPavlo/GazeLandmarksHourGlass/blob/main/eye_landmarks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

from google.colab import drive
drive.mount('/content/gdrive')
ROOT_DATA = 'gdrive/My Drive/unityeye/'

Mounted at /content/gdrive


In [ ]:
import os, sys

!git clone https://github.com/ZPavlo/GazeLandmarksHourGlass 
sys.path.append("GazeLandmarksHourGlass")

Cloning into 'GazeLandmarksHourGlass'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 60 (delta 23), reused 50 (delta 16), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [ ]:
!pip install progress

  Created wheel for progress: filename=progress-1.5-cp36-none-any.whl size=8074 sha256=f4ee6dfb1500da526d3e3bf3a9839c7bb1e9b86737e0876bb715b44d4b536290
  Stored in directory: /root/.cache/pip/wheels/6c/c8/80/32a294e3041f006c661838c05a411c7b7ffc60ff939d14e116
Successfully built progress


In [ ]:
import torch 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from model.hourglass import HourglassNet

NUM_STACKS = 3
NUM_BLOCKS = 4
NUM_CLASSES = 17
print("==> creating model: stacks={}, blocks={}".format(NUM_STACKS, NUM_BLOCKS))
model = HourglassNet(num_stacks=NUM_STACKS, num_blocks=NUM_BLOCKS, num_classes=NUM_CLASSES)
model = model.to(device)

==> creating model: stacks=3, blocks=4


In [ ]:
from losses import JointsMSELoss
from utils.optimizer import get_optimizer

LR = 0.0001

criterion = JointsMSELoss().to(device)
optimizer = get_optimizer("adam", model, lr=LR)

In [ ]:
from utils.data_preprocessing import EyeLandmarksDataset, TrainDataset, TestDataset
from torch.utils.data import DataLoader

unity_eye = EyeLandmarksDataset(ROOT_DATA, load_full=True)
train_dataset = TrainDataset(unity_eye, data_config={
    "max_shift": (5, 7),
    "delta_scale": 0.2,
    "max_rotation_angle": 0.5,
    "image_size": (128, 96),
    "line_count": 2,
    "down_up_scale": 0.4,
    "sigma_head_map": 35.0,
})

test_dataset = TestDataset(unity_eye, {
    "line_count": 2,
    "image_size": (128, 96),
})

100%|██████████| 10/10 [02:14<00:00, 13.48s/it]

Max shift: (5, 7)
Delta scale: 0.2
Max rotation angle: 0.5
Image size: (128, 96)
Line count: 2
Down up scale: 0.4
Sigma head map: 35.0
Max shift: (3, 4)
Delta scale: 0.0
Max rotation angle: 0.3
Image size: (128, 96)
Line count: 2
Down up scale: 0.0
Sigma head map: 1.0


In [ ]:
from train import train, validate
from utils.tools import adjust_learning_rate
lr = LR

lrs = []
train_losses = []
valid_losses = []
train_acc_list = []
valid_acc_list = []
start_epoch = 0
best_acc = 0.0

In [ ]:
BATCH_SIZE = 16
NUM_WORKERS = 1

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

In [ ]:
EPOCH_SIZE = 10
SNAPSHOT = 2
CHECKPOINT_PATH = r"checkpoints\exp1"

SCHEDULE = [3, 6, 9]
GAMMA = 0.8

for epoch in range(start_epoch, EPOCH_SIZE):
    lr = adjust_learning_rate(optimizer, epoch, lr, SCHEDULE, GAMMA)
    print('\nEpoch: %d | LR: %.8f' % (epoch + 1, lr))

    train_dataset.set_difficult((epoch + 1) / EPOCH_SIZE)
    test_dataset.set_difficult((epoch + 1) / EPOCH_SIZE)
    # train for one epoch
    train_loss, train_acc, best_acc = train(train_dataloader, model, criterion, optimizer, best_acc)

    # evaluate on validation set
    valid_loss, valid_acc, predictions, best_acc = validate(test_dataloader, model, criterion, best_acc)

    # remember best acc and save checkpoint
    is_best = valid_acc > best_acc
    best_acc = max(valid_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, predictions, is_best, checkpoint=CHECKPOINT_PATH, snapshot=SNAPSHOT)



Epoch: 1 | LR: 0.00010000
(1/5313) Data: 0.241085s | Batch: 1.237s | Total: 0:00:01 | ETA: 0:00:00 | Loss: 0.0514 | Acc:  0.0000
(2/5313) Data: 0.002627s | Batch: 0.801s | Total: 0:00:02 | ETA: 1:49:30 | Loss: 0.0513 | Acc:  0.0079
(3/5313) Data: 0.002270s | Batch: 0.870s | Total: 0:00:02 | ETA: 1:30:11 | Loss: 0.0503 | Acc:  0.0053
(4/5313) Data: 0.002293s | Batch: 0.816s | Total: 0:00:03 | ETA: 1:25:47 | Loss: 0.0496 | Acc:  0.0058
(5/5313) Data: 0.002312s | Batch: 0.840s | Total: 0:00:04 | ETA: 1:22:23 | Loss: 0.0490 | Acc:  0.0046
(6/5313) Data: 0.002435s | Batch: 0.843s | Total: 0:00:05 | ETA: 1:20:45 | Loss: 0.0482 | Acc:  0.0039
(7/5313) Data: 0.002532s | Batch: 0.928s | Total: 0:00:06 | ETA: 1:19:43 | Loss: 0.0476 | Acc:  0.0056
(8/5313) Data: 0.002313s | Batch: 0.824s | Total: 0:00:07 | ETA: 1:20:03 | Loss: 0.0470 | Acc:  0.0058
(9/5313) Data: 0.002447s | Batch: 0.872s | Total: 0:00:08 | ETA: 1:19:08 | Loss: 0.0461 | Acc:  0.0051
(10/5313) Data: 0.002438s | Batch: 0.793s | To

In [ ]:
print(train_losses)
print(valid_losses)
print(train_acc_list)
print(valid_acc_list)